# Colab LoRA Fine-Tuning Tool

This notebook fine-tunes open-source chat models (e.g., `meta-llama/Llama-2-7b-chat-hf`, `mistralai/Mistral-7B-Instruct`) using standard LoRA adapters without 4-bit quantization. It targets users who have at least a single 16-24 GB GPU available in Google Colab and prefer a simpler, full-precision setup compared to QLoRA.

---

## 📚 What is LoRA? (Low-Rank Adaptation)

### The Problem: Fine-Tuning Large Language Models is Expensive

Traditional fine-tuning updates **all** the parameters of a pre-trained model. For a 7B parameter model like Llama-2:
- **Memory**: You need to store the model weights (~14 GB in fp16), gradients (~14 GB), and optimizer states (~28 GB for Adam). That's **~56 GB** just for training!
- **Storage**: Each fine-tuned model is a full copy (~14 GB per task).
- **Catastrophic forgetting**: Full fine-tuning can overwrite the model's pre-trained knowledge.

### The Solution: LoRA (Low-Rank Adaptation)

**LoRA** was introduced in the paper *"LoRA: Low-Rank Adaptation of Large Language Models"* (Hu et al., 2021). The key insight is:

> **The weight updates during fine-tuning have low intrinsic rank.**

This means we don't need to update all parameters, we can approximate the updates with much smaller matrices!

### 🧮 The Mathematics Behind LoRA

For a pre-trained weight matrix $W_0 \in \mathbb{R}^{d \times k}$, instead of computing:

$$W = W_0 + \Delta W$$

where $\Delta W \in \mathbb{R}^{d \times k}$ has $d \times k$ trainable parameters, LoRA decomposes the update as:

$$W = W_0 + BA$$

where:
- $B \in \mathbb{R}^{d \times r}$ (down-projection)
- $A \in \mathbb{R}^{r \times k}$ (up-projection)
- $r \ll \min(d, k)$ is the **rank** (typically 8-64)

**Parameter reduction example:**
- Original: $d \times k = 4096 \times 4096 = 16.7M$ parameters
- LoRA with $r=16$: $(d \times r) + (r \times k) = 4096 \times 16 + 16 \times 4096 = 131K$ parameters
- **Reduction: 99.2%!**

### 🔧 How LoRA Works During Training

```
                    ┌─────────────────┐
    Input x ───────►│  W₀ (frozen)    │──────┐
                    └─────────────────┘      │
                                             ▼
                    ┌─────┐   ┌─────┐      ┌───┐
    Input x ───────►│  A  │──►│  B  │──────►│ + │────► Output
                    └─────┘   └─────┘      └───┘
                    (r×k)     (d×r)
                    trainable trainable
```

1. **Freeze the original weights** $W_0$ ,  these don't receive gradients
2. **Initialize A** with random Gaussian values
3. **Initialize B** with zeros ,  so $BA = 0$ at the start (no change to pre-trained behavior)
4. **Scale the output** by $\alpha/r$ where $\alpha$ is `lora_alpha`

The scaling factor $\alpha/r$ controls the magnitude of the LoRA update relative to the original weights.

### 📊 Key LoRA Hyperparameters

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| `r` (rank) | Rank of the low-rank matrices. Higher = more capacity but more parameters | 8, 16, 32, 64 |
| `lora_alpha` | Scaling factor. The update is scaled by `alpha/r` | 16, 32 (often 2× rank) |
| `lora_dropout` | Dropout applied to LoRA layers for regularization | 0.0 - 0.1 |
| `target_modules` | Which layers to apply LoRA to | Attention (q,k,v,o) and MLP layers |

### 🎯 Which Layers to Target?

Research shows that applying LoRA to **attention layers** is most effective:
- `q_proj`, `k_proj`, `v_proj`, `o_proj` ,  Query, Key, Value, and Output projections

For even better results, also target **MLP layers**:
- `gate_proj`, `up_proj`, `down_proj` ,  Feed-forward network projections

### ✅ Advantages of LoRA

1. **Memory efficient**: Only train ~0.1-1% of parameters
2. **Fast training**: Fewer gradients to compute
3. **Small adapters**: Save/share just the LoRA weights (~10-50 MB)
4. **Composable**: Switch adapters at inference time for different tasks
5. **No inference latency**: Merge LoRA weights into base model for deployment

### ⚠️ LoRA vs Full Fine-Tuning

| Aspect | Full Fine-Tuning | LoRA |
|--------|------------------|------|
| Trainable params | 100% | ~0.1-1% |
| GPU memory | Very high | Moderate |
| Training speed | Slow | Fast |
| Storage per task | Full model | Small adapter |
| Quality | Best possible | Near full fine-tuning |


---

## 🖥️ Runtime Checklist

1. Go to **Runtime ▸ Change runtime type** and choose `T4` (free) or `L4/A100` (Colab Pro/Pro+).
2. Ensure the **Hardware accelerator** is set to GPU.
3. After connecting, run the cell below to confirm that the GPU is visible to PyTorch.

> Full-precision LoRA consumes more VRAM than QLoRA. If you only have ~16 GB, prefer 7B models with modest batch sizes.

### 💾 Memory Requirements for Full-Precision LoRA

| Model Size | Model Weights (bf16) | Gradients | Optimizer States | Total (Approx.) |
|------------|---------------------|-----------|------------------|-----------------|
| 7B | ~14 GB | ~14 GB | ~28 GB | ~56 GB |
| **7B + LoRA** | **~14 GB** | **~0.1 GB** | **~0.2 GB** | **~15 GB** |

With LoRA, we only compute gradients for the small adapter matrices, dramatically reducing memory usage!


In [ ]:
!nvidia-smi


### 📦 Installing Dependencies

The cell below installs the core libraries:
- **`transformers`**: Hugging Face's library for loading and using pre-trained models
- **`peft`**: Parameter-Efficient Fine-Tuning library (implements LoRA)
- **`trl`**: Transformer Reinforcement Learning library (provides `SFTTrainer` for supervised fine-tuning)
- **`datasets`**: For loading and processing training data
- **`accelerate`**: Handles distributed training and mixed precision
- **`wandb`**: Weights & Biases for experiment tracking (optional)

In [ ]:
%%capture
%pip install -U accelerate==0.30.1 datasets==2.19.1 evaluate==0.4.2 huggingface_hub==0.24.5 peft==0.11.1 sentencepiece==0.1.99 transformers==4.44.2 trl==0.9.4 wandb


In [ ]:
from huggingface_hub import login

login(token=input("Paste your Hugging Face access token: ").strip(), add_to_git_credential=True)


---

## 📂 Data Ingestion Options

- **Upload `.txt`/`.json` files** to Colab and point `cfg.text_folder` to `/content/data`.
- **Mount Google Drive** for larger corpora:
  ```python
  from google.colab import drive
  drive.mount('/content/drive')
  ```
- **Reference a Hugging Face dataset** (e.g., `tatsu-lab/alpaca`) by setting `cfg.dataset_source="hf_dataset"`.

The helper below normalizes data into `instruction`, `response`, and optional `system` fields before training.

### 🎓 Understanding the Training Data Format

For instruction-tuning (teaching the model to follow instructions), we use a structured format:

| Field | Description | Example |
|-------|-------------|---------|
| `instruction` | The task/question for the model | "Summarize the following text:" |
| `response` | The expected output | "The text discusses..." |
| `system` | (Optional) System prompt defining the model's persona | "You are a helpful assistant." |

### 📊 The Config Class Explained

The configuration below controls all aspects of training:


In [ ]:
from dataclasses import asdict, dataclass
from pathlib import Path


@dataclass
class Config:
    project_name: str = "lora-full-precision"
    base_model: str = "meta-llama/Llama-2-7b-chat-hf"
    dataset_source: str = "text_folder"  # text_folder | hf_dataset
    text_folder: str = "/content/data"
    hf_dataset: str | None = None
    max_samples: int | None = None
    chunk_tokens: int = 1024
    chunk_overlap: int = 128
    system_prompt: str = "You are a helpful assistant."
    output_dir: str = "/content/lora-output"
    wandb_project: str | None = None

    micro_batch_size: int = 1
    gradient_accumulation_steps: int = 8
    epochs: float = 3.0
    learning_rate: float = 1e-4
    warmup_ratio: float = 0.03
    weight_decay: float = 0.0
    cutoff_len: int = 2048
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    seed: int = 42


cfg = Config()
print(asdict(cfg))


### 🔍 Config Parameters Deep Dive

**Model & Data Settings:**
- `base_model`: The pre-trained model to fine-tune (e.g., Llama-2-7b-chat)
- `chunk_tokens` / `chunk_overlap`: For long documents, we split them into overlapping chunks

**LoRA Hyperparameters:**
- `lora_r=16`: The rank of the low-rank matrices. Lower = fewer parameters but less capacity
- `lora_alpha=32`: Scaling factor. The update is multiplied by `alpha/r = 32/16 = 2`
- `lora_dropout=0.05`: Regularization to prevent overfitting

**Training Hyperparameters:**
- `micro_batch_size=1`: Samples per GPU per step (keep low for memory)
- `gradient_accumulation_steps=8`: Effective batch size = 1 × 8 = 8
- `learning_rate=1e-4`: LoRA typically uses higher LR than full fine-tuning
- `cutoff_len=2048`: Maximum sequence length (longer = more memory)

### 📈 Data Processing Pipeline

The code below implements:
1. **Text normalization**: Remove extra whitespace, standardize formatting
2. **Chunking**: Split long documents into overlapping segments
3. **Format conversion**: Transform various data formats into our standard schema

In [ ]:
import random
import re

import pandas as pd
from datasets import Dataset, load_dataset


def _normalize(text: str) -> str:
    text = text.replace("\r", " ").strip()
    return re.sub(r"\s+", " ", text)


def _chunk_words(text: str, chunk_tokens: int, overlap: int) -> list[str]:
    words = text.split()
    if not words:
        return []
    step = max(chunk_tokens - overlap, 1)
    chunks = []
    for start in range(0, len(words), step):
        segment = words[start:start + chunk_tokens]
        if len(segment) < 32:
            continue
        chunks.append(" ".join(segment))
    return chunks or [" ".join(words[:chunk_tokens])]


def _load_local_texts(folder: str, cfg: Config) -> list[dict]:
    folder_path = Path(folder)
    rows = []
    for path in folder_path.rglob("*.txt"):
        text = path.read_text(encoding="utf-8")
        for chunk in _chunk_words(_normalize(text), cfg.chunk_tokens, cfg.chunk_overlap):
            rows.append({
                "instruction": f"Answer using {path.stem} context.",
                "response": chunk,
                "system": cfg.system_prompt,
            })
    return rows


def _load_hf_dataset(repo_id: str, cfg: Config) -> list[dict]:
    ds = load_dataset(repo_id, split="train")
    rows = []
    for row in ds:
        if {"instruction", "output"}.issubset(ds.column_names):
            instruction = row["instruction"]
            response = row["output"]
        else:
            instruction = row.get("instruction", "Summarize the passage:")
            response = row.get("text", row.get("response", ""))
        rows.append({
            "instruction": _normalize(str(instruction)),
            "response": _normalize(str(response)),
            "system": row.get("system", cfg.system_prompt),
        })
    return rows


def build_dataset(cfg: Config) -> Dataset:
    if cfg.dataset_source == "hf_dataset" and cfg.hf_dataset:
        rows = _load_hf_dataset(cfg.hf_dataset, cfg)
    else:
        rows = _load_local_texts(cfg.text_folder, cfg)

    if cfg.max_samples:
        random.seed(cfg.seed)
        rows = random.sample(rows, min(cfg.max_samples, len(rows)))

    rows = [r for r in rows if r["instruction"].strip() and r["response"].strip()]
    dataset = Dataset.from_pandas(pd.DataFrame(rows))
    print(f"Dataset has {len(dataset)} rows after cleaning")
    return dataset


dataset = build_dataset(cfg)
dataset[:2]


---

## 💬 Prompt Templating

We build chat prompts using the tokenizer's `chat_template` when available so that LoRA adapters stay aligned with the base instruction format.

### 🎯 Why Prompt Templates Matter

Different models expect different prompt formats. Using the wrong format leads to poor results!

**Llama-2 Chat format:**
```
<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

What is the capital of France? [/INST] The capital of France is Paris. </s>
```

**Mistral Instruct format:**
```
<s>[INST] What is the capital of France? [/INST] The capital of France is Paris.</s>
```

The `chat_template` in the tokenizer automatically handles these differences, ensuring your fine-tuned model uses the correct format.


In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


def build_prompt(example: dict) -> dict:
    messages = [
        {"role": "system", "content": example.get("system") or cfg.system_prompt},
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]},
    ]
    if tokenizer.chat_template:
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    else:
        prompt = (
            f"[SYSTEM]\n{messages[0]['content']}\n\n"
            f"[USER]\n{messages[1]['content']}\n\n"
            f"[ASSISTANT]\n{messages[2]['content']}"
        )
    return {"text": prompt}


processed_dataset = dataset.map(build_prompt, remove_columns=dataset.column_names)
processed_dataset = processed_dataset.shuffle(seed=cfg.seed)
splits = processed_dataset.train_test_split(test_size=0.05, seed=cfg.seed)
splits


---

## 🚀 LoRA Training

Without quantization, we load the base model in bf16/fp16, attach LoRA adapters to attention/MLP modules, and fine-tune via `trl.SFTTrainer`. Keep `micro_batch_size` low to stay within VRAM limits.

### 🔧 Training Pipeline Breakdown

```
┌─────────────────────────────────────────────────────────────────┐
│                        TRAINING FLOW                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Load Base Model (bf16)                                      │
│     └─► 7B params loaded, ~14GB VRAM                           │
│                                                                 │
│  2. Enable Gradient Checkpointing                               │
│     └─► Trade compute for memory (recompute activations)       │
│                                                                 │
│  3. Attach LoRA Adapters                                        │
│     └─► Add small trainable matrices to target layers          │
│                                                                 │
│  4. Freeze Base Weights                                         │
│     └─► Only LoRA parameters receive gradients                 │
│                                                                 │
│  5. Train with SFTTrainer                                       │
│     └─► Supervised Fine-Tuning on instruction-response pairs   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 📋 Key Components Explained

**`LoraConfig` Parameters:**
```python
LoraConfig(
    r=16,                    # Rank: size of low-rank matrices
    lora_alpha=32,           # Scaling: output multiplied by alpha/r
    target_modules=[...],    # Which layers to adapt
    lora_dropout=0.05,       # Regularization
    bias="none",             # Don't train bias terms
    task_type="CAUSAL_LM",   # Language modeling task
)
```

**`SFTTrainer` Features:**
- Handles tokenization and padding automatically
- `packing=True`: Concatenates short sequences to fill context length (more efficient)
- Computes loss only on assistant responses (not system/user prompts)

### ⚡ Memory Optimization Techniques Used

1. **Gradient Checkpointing**: Recomputes activations during backward pass instead of storing them
2. **Mixed Precision (bf16)**: Uses 16-bit floats for most operations
3. **LoRA**: Only stores gradients for ~0.1% of parameters


In [ ]:
import math
import os

import torch
from peft import LoraConfig
from transformers import AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer

os.makedirs(cfg.output_dir, exist_ok=True)
if cfg.wandb_project:
    os.environ["WANDB_PROJECT"] = cfg.wandb_project
else:
    os.environ["WANDB_DISABLED"] = "true"

torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    torch_dtype=torch_dtype,
    device_map="auto",
)
model.gradient_checkpointing_enable()
model.config.use_cache = False

peft_config = LoraConfig(
    r=cfg.lora_r,
    lora_alpha=cfg.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=cfg.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    per_device_train_batch_size=cfg.micro_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    num_train_epochs=cfg.epochs,
    learning_rate=cfg.learning_rate,
    warmup_ratio=cfg.warmup_ratio,
    weight_decay=cfg.weight_decay,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
    bf16=torch_dtype == torch.bfloat16,
    fp16=torch_dtype == torch.float16,
    max_grad_norm=0.3,
    report_to=([] if os.environ.get("WANDB_DISABLED") == "true" else ["wandb"]),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    peft_config=peft_config,
    train_dataset=splits["train"],
    eval_dataset=splits["test"],
    dataset_text_field="text",
    max_seq_length=cfg.cutoff_len,
    packing=True,
    args=training_args,
)

trainer.train()
trainer.save_model(cfg.output_dir)
tokenizer.save_pretrained(cfg.output_dir)


### 📊 Evaluating the Model: Perplexity

**Perplexity** measures how "surprised" the model is by the test data. Lower is better!

$$\text{Perplexity} = e^{\text{Cross-Entropy Loss}}$$

**Interpretation:**
- Perplexity of 10 means the model is "as confused as if it had to choose uniformly between 10 options"
- A well-trained model on domain-specific data typically achieves perplexity of 5-20
- Base models on random text: perplexity 20-100+

In [ ]:
from torch.utils.data import DataLoader


def compute_perplexity(eval_dataset, max_batches: int = 32) -> float:
    model.eval()
    loader = DataLoader(eval_dataset["text"], batch_size=1)
    scores = []
    for idx, batch in enumerate(loader):
        if idx >= max_batches:
            break
        encoded = tokenizer(batch[0], return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**encoded, labels=encoded["input_ids"])
        scores.append(math.exp(outputs.loss.item()))
    return sum(scores) / len(scores)


perplexity = compute_perplexity(splits["test"], max_batches=32)
print(f"Approximate perplexity: {perplexity:.2f}")


### 🗣️ Interactive Testing

The `chat()` function allows you to test your fine-tuned model interactively:
- Uses the same prompt template as training for consistency
- `do_sample=True` enables creative/varied responses
- `top_p=0.9` (nucleus sampling) and `temperature=0.7` control randomness

In [ ]:
def chat(prompt: str, system: str | None = None, max_new_tokens: int = 512) -> str:
    messages = [
        {"role": "system", "content": system or cfg.system_prompt},
        {"role": "user", "content": prompt},
    ]
    template = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(template, return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
    output_tokens = generated[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)


chat("Summarize the three biggest takeaways from our training data.")


### 🔀 Merging LoRA Adapters

After training, you have two deployment options:

**Option 1: Keep Adapters Separate**
- Load base model + LoRA adapter at inference
- Allows hot-swapping different adapters
- Slight overhead for loading adapters

**Option 2: Merge and Unload (shown below)**
- Permanently bake LoRA weights into the base model: $W_{merged} = W_0 + BA$
- Results in a standard model (no PEFT dependency needed)
- No runtime overhead
- Larger storage (full model size)

```
Before Merge:                    After Merge:
┌─────────┐   ┌─────────┐       ┌─────────────────┐
│ Base W₀ │ + │ LoRA BA │  ==>  │ W₀ + BA (merged)│
└─────────┘   └─────────┘       └─────────────────┘
  ~14 GB        ~50 MB                ~14 GB
```

In [ ]:
from peft import AutoPeftModelForCausalLM

MERGED_DIR = Path(cfg.output_dir) / "merged"
MERGED_DIR.mkdir(parents=True, exist_ok=True)

merged_model = AutoPeftModelForCausalLM.from_pretrained(cfg.output_dir, device_map="auto")
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)
print("Merged checkpoint saved to", MERGED_DIR)

push_to_hub = False
if push_to_hub:
    repo_id = input("Target HF repo (username/project): ")
    merged_model.push_to_hub(repo_id, private=True)
    tokenizer.push_to_hub(repo_id, private=True)


---

## 🎯 Next Steps & Best Practices

### 📝 Workflow Recommendations
- Adjust the config block (`cfg`) for your dataset, model, and hyperparameters before running
- Monitor GPU memory via `nvidia-smi` while training; reduce `micro_batch_size` or `cutoff_len` if you hit OOM
- Evaluate the resulting adapter on task-specific prompts and log findings (W&B, TensorBoard, etc.)

### 🚀 Deployment Options

| Option | Description | When to Use |
|--------|-------------|-------------|
| **Separate adapter** | Load base + `PeftModel.from_pretrained` | Multi-task, adapter swapping |
| **Merged checkpoint** | Single model, no PEFT dependency | Simpler deployment |
| **vLLM/TGI** | Production serving with LoRA support | High-throughput APIs |

### 🔧 Troubleshooting Common Issues

| Issue | Solution |
|-------|----------|
| OOM during training | Reduce `micro_batch_size`, enable gradient checkpointing |
| Loss plateaus | Try higher `learning_rate` or larger `lora_r` |
| Overfitting | Increase `lora_dropout`, reduce epochs, add more data |
| Poor generation | Verify prompt template matches training format |

### 📊 LoRA vs QLoRA: When to Use Which?

| Choose **LoRA** when: | Choose **QLoRA** when: |
|-----------------------|------------------------|
| You have ≥24GB VRAM | Limited GPU memory (<16GB) |
| Maximum quality matters | Memory efficiency is priority |
| Faster training needed | Training larger models (13B+) |
| Simpler setup preferred | Free Colab T4 runtime |

### 📚 Further Reading
- [LoRA Paper](https://arxiv.org/abs/2106.09685) - Original research by Microsoft
- [PEFT Documentation](https://huggingface.co/docs/peft) - Hugging Face implementation
- [TRL Documentation](https://huggingface.co/docs/trl) - Training library details


---

## 📖 Summary: Key Concepts

### 🧠 LoRA Essentials

1. **Low-Rank Decomposition**: Instead of updating $W$ directly, decompose the update as $\Delta W = BA$ where $B$ and $A$ are small matrices
2. **Parameter Efficiency**: Train only 0.1-1% of parameters while achieving near full fine-tuning quality
3. **Memory Savings**: No gradients for frozen base weights = massive memory reduction
4. **Composability**: Multiple LoRA adapters can be trained and swapped at inference time

### 🔢 The Math That Matters

$$W_{new} = W_{frozen} + \frac{\alpha}{r} \cdot B \cdot A$$

- $r$ (rank): Controls adapter capacity
- $\alpha$: Controls the magnitude of the update
- $\alpha/r$: The effective scaling factor

### ✅ Quick Reference

| Task | Recommended `r` | Notes |
|------|-----------------|-------|
| Simple adaptation | 8-16 | Fast, lightweight |
| Complex tasks | 32-64 | More capacity |
| Full performance | 128+ | Approaches full fine-tuning |

---
*Happy fine-tuning! 🚀*